In [43]:
%matplotlib inline

import torch
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim

In [75]:
# Hyper parameters
num_epochs = 1
batchsize = 10
learning_rate = 0.001
momentum = 0.5
log_interval = 200
TRAIN_DATA_PATH = "../Data/CATS_DOGS/train"
#TEST_DATA_PATH = "ImageFolder/images/test/"

TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(128),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
    ])

train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM_IMG)
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batchsize, shuffle=True,  num_workers=4)

#test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=TRANSFORM_IMG)
#test_data_loader  = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4) 



In [76]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 10, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = torch.nn.Dropout2d()
        self.fc1 = torch.nn.Linear(16820, 50)
        self.fc2 = torch.nn.Linear(50, 1)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 16820)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.sigmoid(x)

In [77]:
#Network model
network = Network()

#Stochastic gradient decent
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

criterion = torch.nn.BCELoss()

In [74]:
##Just for testign

test = iter(train_data_loader)
img, b= next(test)

out = network(img)

#loss = F.nll_loss(out, b)
a = b.view(-1, 1)
b.float().dtype

torch.float32

In [79]:
#train

for epoch in range(1, num_epochs+1):
    
    for batch_idx, (data, target) in enumerate(train_data_loader):
        
        #print(f'b {batch}')
        
        #forward pass
        out = network(data)
        
        #Use negative log likelihood loss.
        loss = criterion(out, target.view(-1, 1).float())
        
        #with this gradients are calculated
        loss.backward()
    
        #update gradients
        optimizer.step()

        #Display iteration statistics
        if batch_idx % log_interval == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_data_loader.dataset),
            100. * batch_idx / len(train_data_loader), loss.item()))
    
    

ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 

RuntimeError: DataLoader worker (pid(s) 4158) exited unexpectedly

### ToDo

- There are errors during training related with memory, try to increase memory in docker or reduce size of images.